In [1]:
# %%
import matplotlib.dates as mdates
import geopandas as gpd
import pandas as pd
import xarray as xr
import numpy as np
import scipy as sp
import regionmask
import calendar
import pickle
import socket
import dask
import glob
import time
import math
import sys
import ast
import os

ce_outputs_parent_dir = '/nfs/turbo/seas-mtcraig/MacroCEMResults/HariPaper1Mar16/'
met_dir = '/nfs/turbo/seas-mtcraig/data_sharing/hari_paper1_ERA5/'

# %%
# # Dask
#from dask_mpi import initialize
#initialize()

from distributed import Client
import dask.array as da

client = Client(n_workers=12)

print("Here")

def get_field_timeseries_latlon(dset:xr.Dataset, var_name:str, 
                                lat:float, lon:float,
                                lat_name:str="lat", lon_name:str="lon"
                               ) -> xr.DataArray:
    """
    Find the time series of a required field closest to a given lat,lon point.
    Required for edge cases where the dataset has been masked by a shapefile, but 
    xarray retains the rectilinear grid, so the given point might be close to a gridpoint
    that has NaN value. 

    Args:
        dset (xr.Dataset): Dataset containing required field. (lat,lon,time) dims
        var_name (str): Required field name
        lat (float): Required lat
        lon (float): Required lon
        lat_name (str, optional): Dataset might have latitude or lat or other name. 
                                  Defaults to "lat".
        lon_name (str, optional): Dataset might have longitude or lon or other name. 
                                  Defaults to "lon".
    Returns:
        xr.DataArray: Time series of required field
    """
   
    if(np.isnan(lat) or np.isnan(lon)):
        latCenter,lonCenter = dset[lat_name][int(dset.dims[lat_name]/2)].values,dset[lon_name][int(dset.dims[lon_name]/2)].values
        reqd_field_ts = dset[var_name].sel({lat_name:latCenter,lon_name:lonCenter},method='nearest')
        return reqd_field_ts
    
    latCenter,lonCenter = dset[lat_name][int(dset.dims[lat_name]/2)].values,dset[lon_name][int(dset.dims[lon_name]/2)].values
    latStep,lonStep = abs(dset[lat_name][0].values-dset[lat_name][1].values),abs(dset[lon_name][0].values-dset[lon_name][1].values)
    
    while np.isnan(dset.isel(time=-1).sel({lat_name:lat,lon_name:lon},method='nearest').compute()[var_name]):
        lat = lat + ((latCenter-lat)/((latCenter-lat)**2+(lonCenter-lon)**2)**0.5)*latStep
        lon = lon + ((lonCenter-lon)/((latCenter-lat)**2+(lonCenter-lon)**2)**0.5)*lonStep

    reqd_field_ts = dset[var_name].sel({lat_name:lat,lon_name:lon},method='nearest')

    return reqd_field_ts

Here


In [2]:
client.scheduler_info()

{'type': 'Scheduler',
 'id': 'Scheduler-6a69e778-e213-4170-8947-9efce9e4ea64',
 'address': 'tcp://127.0.0.1:37917',
 'services': {'dashboard': 8787},
 'started': 1682868104.9762235,
 'workers': {'tcp://127.0.0.1:34071': {'type': 'Worker',
   'id': 6,
   'host': '127.0.0.1',
   'resources': {},
   'local_directory': '/home/sriharis/paper1/revision_stuff/modify_fleet/get_2_4_2017/dask-worker-space/worker-9i4atv5f',
   'name': 6,
   'nthreads': 3,
   'memory_limit': 16106127360,
   'last_seen': 1682868138.5336778,
   'services': {'dashboard': 34841},
   'metrics': {'executing': 0,
    'in_memory': 0,
    'ready': 0,
    'in_flight': 0,
    'bandwidth': {'total': 100000000, 'workers': {}, 'types': {}},
    'spilled_nbytes': 0,
    'cpu': 0.0,
    'memory': 80302080,
    'time': 1682868138.512781,
    'read_bytes': 5249250.449574727,
    'write_bytes': 2385096.3207776425,
    'read_bytes_disk': 0.0,
    'write_bytes_disk': 0.0,
    'num_fds': 20},
   'nanny': 'tcp://127.0.0.1:33109'},
  'tcp://127.0.0.1:34441': {'type': 'Worker',
   'id': 10,
   'host': '127.0.0.1',
   'resources': {},
   'local_directory': '/home/sriharis/paper1/revision_stuff/modify_fleet/get_2_4_2017/dask-worker-space/worker-f7pkz2gu',
   'name': 10,
   'nthreads': 3,
   'memory_limit': 16106127360,
   'last_seen': 1682868138.5368905,
   'services': {'dashboard': 44861},
   'metrics': {'executing': 0,
    'in_memory': 0,
    'ready': 0,
    'in_flight': 0,
    'bandwidth': {'total': 100000000, 'workers': {}, 'types': {}},
    'spilled_nbytes': 0,
    'cpu': 0.0,
    'memory': 80404480,
    'time': 1682868138.5116277,
    'read_bytes': 1279363.062200957,
    'write_bytes': 496693.8947368421,
    'read_bytes_disk': 0.0,
    'write_bytes_disk': 0.0,
    'num_fds': 20},
   'nanny': 'tcp://127.0.0.1:40095'},
  'tcp://127.0.0.1:38367': {'type': 'Worker',
   'id': 8,
   'host': '127.0.0.1',
   'resources': {},
   'local_directory': '/home/sriharis/paper1/revision_stuff/modify_fleet/get_2_4_2017/dask-worker-space/worker-zv7dpuws',
   'name': 8,
   'nthreads': 3,
   'memory_limit': 16106127360,
   'last_seen': 1682868138.5376165,
   'services': {'dashboard': 33991},
   'metrics': {'executing': 0,
    'in_memory': 0,
    'ready': 0,
    'in_flight': 0,
    'bandwidth': {'total': 100000000, 'workers': {}, 'types': {}},
    'spilled_nbytes': 0,
    'cpu': 0.0,
    'memory': 80330752,
    'time': 1682868138.5126228,
    'read_bytes': 3934564.990825688,
    'write_bytes': 1616153.8348623854,
    'read_bytes_disk': 0.0,
    'write_bytes_disk': 0.0,
    'num_fds': 20},
   'nanny': 'tcp://127.0.0.1:38425'},
  'tcp://127.0.0.1:38487': {'type': 'Worker',
   'id': 2,
   'host': '127.0.0.1',
   'resources': {},
   'local_directory': '/home/sriharis/paper1/revision_stuff/modify_fleet/get_2_4_2017/dask-worker-space/worker-3g3x0wij',
   'name': 2,
   'nthreads': 3,
   'memory_limit': 16106127360,
   'last_seen': 1682868138.331382,
   'services': {'dashboard': 35777},
   'metrics': {'executing': 0,
    'in_memory': 0,
    'ready': 0,
    'in_flight': 0,
    'bandwidth': {'total': 100000000, 'workers': {}, 'types': {}},
    'spilled_nbytes': 0,
    'cpu': 2.0,
    'memory': 83542016,
    'time': 1682868138.3305106,
    'read_bytes': 12655.354024146736,
    'write_bytes': 8464.839446614704,
    'read_bytes_disk': 0.0,
    'write_bytes_disk': 0.0,
    'num_fds': 23},
   'nanny': 'tcp://127.0.0.1:41231'},
  'tcp://127.0.0.1:39047': {'type': 'Worker',
   'id': 7,
   'host': '127.0.0.1',
   'resources': {},
   'local_directory': '/home/sriharis/paper1/revision_stuff/modify_fleet/get_2_4_2017/dask-worker-space/worker-a17t2fm4',
   'name': 7,
   'nthreads': 3,
   'memory_limit': 16106127360,
   'last_seen': 1682868138.5343826,
   'services': {'dashboard': 35691},
   'metrics': {'executing': 0,
    'in_memory': 0,
    'ready': 0,
    'in_flight': 0,
    'bandwidth': {'total': 100000000, 'workers': {}, 'types': {}},
    'spilled_nbytes': 0,
    'cpu': 0.0,
    'memory': 80297984,
    'time': 1682

In [3]:
# %%
from RAM import *

In [4]:
# %% [markdown]
# ### Pyomo flow optim stuff

# %%
import scipy.optimize as spopt
import scipy as sp
import numpy as np
import math
import time
from pyomo.opt import SolverStatus, TerminationCondition
import pyomo.environ as pyo


def get_dicts_from_numpy(nparray):

    if nparray.ndim == 1:
        array_dict = {(i+1):val for i,val in enumerate(nparray)}

    elif nparray.ndim == 2:
        array_dict = {(i+1,j+1):nparray[i,j] \
                      for j in range(nparray.shape[1])\
                      for i in range(nparray.shape[0])}

    else:
        return

    return array_dict

flow_cost = 5
ens_cost = 100

def init_transmission(model,max_flows):            
    all_regions = ['CAMX', 'Desert_Southwest', 'NWPP_Central', 'NWPP_NE', 'NWPP_NW']
    N = len(all_regions)
    nodes = np.arange(N)

    edges = np.arange(N*(N-1))

    edge_assoc = []
    for i in range(N):
        for j in range(N):
            if i!=j:
                edge_assoc.append((i,j))

    nodal_constraint = np.zeros((N,N*(N-1)))

    for i in range(N):
        for j in range(N*(N-1)):
            if i == edge_assoc[j][0]:
                nodal_constraint[i,j] = 1
            elif i == edge_assoc[j][1]:
                nodal_constraint[i,j] = -1

    model.nodes = pyo.RangeSet(N)
    model.edges = pyo.RangeSet(N*(N-1))

    model.max_flows = pyo.Param(model.edges, initialize=get_dicts_from_numpy(max_flows))
    model.nodal_constraint_matrix = pyo.Param(model.nodes, model.edges, 
                                              initialize=get_dicts_from_numpy(nodal_constraint))
    
    return model

def o_rule(model):
    return model.flow_cost*sum(model.flows[i] for i in model.edges) + model.ens_cost*sum(model.unserved_energy[j] for j in model.nodes) 

def nodal_residual_cons_rule(model,i):
    return (sum(model.nodal_constraint_matrix[i,j]*model.flows[j] for j in model.edges) 
            - model.unserved_energy[i] <= model.initial_residuals[i])
    
def flow_constraint_rule(model,i):
    return (model.flows[i] <= model.max_flows[i])

def init_model(max_flows):
    model = pyo.AbstractModel()
    
    model = init_transmission(model,max_flows)
    
    model.initial_residuals = pyo.Param(model.nodes)
    model.flows = pyo.Var(model.edges, domain=pyo.NonNegativeReals)
    model.unserved_energy = pyo.Var(model.nodes, domain=pyo.NonNegativeReals)

    model.obj = pyo.Objective(rule=o_rule, sense=pyo.minimize)
    model.nodal_residual_constraint = pyo.Constraint(model.nodes,rule=nodal_residual_cons_rule)
    model.flow_constraint = pyo.Constraint(model.edges,rule=flow_constraint_rule)

    model.flow_cost = pyo.Param(initialize=flow_cost)
    model.ens_cost = pyo.Param(initialize=ens_cost)

    return model

def create_instance_getoutput(model,row):
    instance = model.create_instance({None:{'initial_residuals':get_dicts_from_numpy(row)}})

    result = pyo.SolverFactory('gurobi').solve(instance)

    flow = np.array([instance.flows[i]() for i in model.edges])
    ens = np.array([instance.unserved_energy[i]() for i in model.nodes])
    
    return flow,ens


In [5]:
def get_subregional_fleets(fleet_df,wecc_demand_year):    
    
    reqd_cols = ['PlantType','Latitude','Longitude','Nameplate Energy Capacity (MWh)','Capacity (MW)','region']

    # Rename new RE plant types to same name as existing plants
    fleet_df = fleet_df[fleet_df['Retired'].isin([False])][reqd_cols]
    fleet_df.loc[fleet_df[fleet_df['PlantType'].str.match('windlat')]['PlantType'].index,'PlantType'] = 'Onshore Wind'
    fleet_df.loc[fleet_df[fleet_df['PlantType'].str.match('solarlat')]['PlantType'].index,'PlantType'] = 'Solar PV'

    # Rename plant types to match FOR dict
    fleet_df.loc[fleet_df[fleet_df['PlantType'].str.match('Solar')]['PlantType'].index,'PlantType'] = 'Solar'
    fleet_df.loc[fleet_df[fleet_df['PlantType'].str.match('Onshore')]['PlantType'].index,'PlantType'] = 'Wind'
    fleet_df.loc[fleet_df[fleet_df['PlantType'].str.match('Combined')]['PlantType'].index,'PlantType'] = 'CC'
    fleet_df.loc[fleet_df[fleet_df['PlantType'].str.match('Combust')]['PlantType'].index,'PlantType'] = 'CT'
    fleet_df.loc[fleet_df[fleet_df['PlantType'].str.match('Coal')]['PlantType'].index,'PlantType'] = 'ST'
    fleet_df.loc[fleet_df[fleet_df['PlantType'].str.match('Hydro')]['PlantType'].index,'PlantType'] = 'HD'
    fleet_df.loc[fleet_df[fleet_df['PlantType'].str.match('Nuclear')]['PlantType'].index,'PlantType'] = 'NU'
    fleet_df.loc[fleet_df[fleet_df['PlantType'].str.match('Pumped Storage')]['PlantType'].index,'PlantType'] = 'Storage'
    fleet_df.loc[fleet_df[fleet_df['PlantType'].str.match('Energy Storage')]['PlantType'].index,'PlantType'] = 'Storage'
    fleet_df.loc[fleet_df[fleet_df['PlantType'].str.match('Battery')]['PlantType'].index,'PlantType'] = 'Storage'
    fleet_df.loc[fleet_df[fleet_df['PlantType'].str.match('Batteries')]['PlantType'].index,'PlantType'] = 'Storage'
    fleet_df.loc[fleet_df[fleet_df['PlantType'].str.match('Cell')]['PlantType'].index,'PlantType'] = 'Storage'
    fleet_df.loc[fleet_df[~(fleet_df['PlantType'].isin(['NU','HD','ST','CT','CC','Solar','Wind','Storage']))].index,'PlantType'] = 'Other'

    solar_cf = xr.open_dataset(met_dir+'wecc_solarCF_ERA5_hourly_PST.nc').sel(time=reqd_slice_year)
    wind_cf = xr.open_dataset(met_dir+'wecc_windCF_ERA5_hourly_PST.nc').sel(time=reqd_slice_year)
    temp = xr.open_dataset(met_dir+'wecc_temp_ERA5_hourly_PST.nc').sel(time=reqd_slice_year)
    forced_outages = xr.open_dataset(met_dir+'wecc_FOR_ERA5_hourly_PST.nc').sel(time=reqd_slice_year)
    
    #### Hourly capacities
    all_regions = ['CAMX', 'Desert_Southwest', 'NWPP_Central', 'NWPP_NE', 'NWPP_NW']
    non_storage_gens = ['NU','HD','ST','CT','CC','Solar','Wind','Other']

    regional_caps_fors = dict()

    for region_here in all_regions:
        
        fleet_df_region = fleet_df.loc[(fleet_df["region"].isin([region_here]))]
        nameplate_region = fleet_df_region.loc[fleet_df_region["PlantType"].isin(non_storage_gens)]['Capacity (MW)'].reset_index(drop=True)
        tech_region = fleet_df_region.loc[fleet_df_region["PlantType"].isin(non_storage_gens)]['PlantType'].reset_index()
        coords_region = fleet_df_region.loc[fleet_df_region["PlantType"].isin(non_storage_gens)][['Latitude','Longitude']].reset_index()

        hourly_capacity_factors = np.ones((len(wecc_demand_year),len(nameplate_region)))
        for_matrix = np.ones((len(wecc_demand_year),len(nameplate_region))) * 0.05

        for gentype in non_storage_gens:
            
            idx_region_gentype = tech_region.loc[tech_region["PlantType"].isin([gentype])].index
            coords_region_gentype = coords_region.loc[idx_region_gentype]

            if gentype == 'Solar':
                arr = np.array([get_field_timeseries_latlon(solar_cf,'Solar_CF',
                                                            row['Latitude'],
                                                            row['Longitude']
                                                        ).values for index,row in coords_region_gentype.iterrows()]
                            )
                hourly_capacity_factors[:,coords_region_gentype.index] = arr.T
                
            elif gentype == 'Wind':
                arr = np.array([get_field_timeseries_latlon(wind_cf,'Wind_CF',
                                                            row['Latitude'],
                                                            row['Longitude']
                                                        ).values for index,row in coords_region_gentype.iterrows()]
                            )

                hourly_capacity_factors[:,coords_region_gentype.index] = arr.T
            
            else:
                arr = np.array([get_field_timeseries_latlon(forced_outages,gentype+'_FOR',
                                                            row['Latitude'],
                                                            row['Longitude']
                                                        ).values for index,row in coords_region_gentype.iterrows()]
                            )
                
                for_matrix[:,coords_region_gentype.index] = arr.T

        hourly_capacity = np.multiply(hourly_capacity_factors,nameplate_region.to_numpy())
        
        # Hydro electric generation
        # Dispatch hydro each hour as long as monthly balance is available, 
        # and there is a net load deficit from CC,solar, and wind

        # Each plant's generation is a fraction of the total hourly generation,
        # where the fraction is calculated based on the nameplate capacity
        idx_region_gentype = tech_region.loc[tech_region["PlantType"].isin(['HD'])].index
        idx_region_gentype_notHD = tech_region.loc[~tech_region["PlantType"].isin(['HD'])].index
        idx_region_gentype_RE = tech_region.loc[tech_region["PlantType"].isin(['Solar','Wind'])].index
        hydro_multip = nameplate_region.loc[idx_region_gentype]
        hydro_multip = hydro_multip/hydro_multip.sum()

        monthly_gen = wecc_hydro_monthlygen.loc[int(year)].to_numpy()
        days_in_month_dict = np.array([31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31])
        hours_till_month = np.append([0],[(days_in_month_dict*24).cumsum()])

        region_hydro_nameplate = nameplate_region.loc[idx_region_gentype].sum() 
        total_hydro_capacity = monthly_gen*region_hydro_nameplate\
                            /fleet_df.loc[fleet_df["PlantType"].isin(['HD'])]['Capacity (MW)'].sum()
        
        hydro_dispatch = np.zeros((8760))
        netload_wo_hydro = wecc_demand_year[region_here].to_numpy() - hourly_capacity[:,idx_region_gentype_notHD].sum(axis=1)
        netload_deficit = np.where(netload_wo_hydro>0,netload_wo_hydro,0)

        netload_RE = wecc_demand_year[region_here].to_numpy() - hourly_capacity[:,idx_region_gentype_RE].sum(axis=1)
        netload_RE = np.where(netload_RE>0,netload_RE,0)
        
        for month in range(12):
            monthly_total_hydro_capacity = total_hydro_capacity[month]
            for hour in range(hours_till_month[month],hours_till_month[month+1]):

                if monthly_total_hydro_capacity < netload_deficit[hour]:
                    print(region_here,hour)
                    hydro_dispatch[hour] = monthly_total_hydro_capacity if monthly_total_hydro_capacity < region_hydro_nameplate \
                                        else region_hydro_nameplate
                    break
                    
                hydro_dispatch[hour] = netload_deficit[hour] if netload_deficit[hour]< region_hydro_nameplate \
                                        else region_hydro_nameplate 
                monthly_total_hydro_capacity -= hydro_dispatch[hour]
                
            if monthly_total_hydro_capacity >= 0:
                
                # smear the remaining hydro in the month proportional 
                # to the netload curve (demand - generation from RE)
                normalize_netload_RE = netload_RE[hours_till_month[month]:hours_till_month[month+1]]
                normalize_netload_RE = normalize_netload_RE/normalize_netload_RE.sum()
                
                spread = monthly_total_hydro_capacity*normalize_netload_RE
                
                dispatch = np.where(spread+hydro_dispatch[hours_till_month[month]:hours_till_month[month+1]]\
                                            <region_hydro_nameplate,
                                    spread+hydro_dispatch[hours_till_month[month]:hours_till_month[month+1]],
                                    region_hydro_nameplate)
                
                
                hydro_dispatch[hours_till_month[month]:hours_till_month[month+1]] = dispatch

        hourly_capacity[:,idx_region_gentype] = hydro_multip.to_numpy()*hydro_dispatch.reshape((-1,1))
         
        for_matrix = da.from_array(for_matrix,chunks=(150,nameplate_region.size))
        hourly_capacity_factors = da.from_array(hourly_capacity_factors,chunks=(150,nameplate_region.size))
        hourly_capacity = da.from_array(hourly_capacity,chunks=(150,nameplate_region.size))

        storage_params = {'round_trip_efficiency': {1: 0.8},
                          'efor':{1:0.05}} 

        storage = initialize_storage_from_df(fleet_df_region[(fleet_df_region['PlantType'].isin(['Storage']))], 
                                             'Capacity (MW)','Nameplate Energy Capacity (MWh)',
                                             storage_params['efor'][1], 
                                             'reliability',storage_params['round_trip_efficiency'][1])

        regional_caps_fors[region_here] = {'nameplate_region':nameplate_region,
                                           'storage_region':storage,
                                           'hourly_capacity':hourly_capacity,
                                           'hourly_capacity_factors':hourly_capacity_factors,
                                           'for_matrix':for_matrix,
                                           'tech_region':tech_region}
        
    return regional_caps_fors

In [6]:
def get_lolp_system_wo_xmission(regional_caps_fors,reqd_region=None):    
    regional_lolp_hourlycaps = dict()

    if reqd_region == None:
        all_regions = ['CAMX', 'Desert_Southwest', 'NWPP_Central', 'NWPP_NE', 'NWPP_NW']
    else:
        all_regions = [reqd_region]

    for region_here in all_regions:
        lolp,hourly_system_capacity = calc_lolp_full(n_iters,
                            regional_caps_fors[region_here]['for_matrix'],
                            regional_caps_fors[region_here]['nameplate_region'],
                            regional_caps_fors[region_here]['hourly_capacity'],
                            wecc_demand_year[region_here].to_numpy(),
                            regional_caps_fors[region_here]['storage_region'],
                            return_hourly_capacity=True
                            )
        
        regional_lolp_hourlycaps[region_here] = (lolp, hourly_system_capacity)

    lolp_df_beforexfers = pd.DataFrame(np.array([regional_lolp_hourlycaps[region_here][0] for region_here in all_regions]).T,
                               index=wecc_demand_year.index,columns=all_regions)
    regional_hourlycaps = {region_here:regional_lolp_hourlycaps[region_here][1] for region_here in all_regions}

    return lolp_df_beforexfers, regional_hourlycaps

In [7]:
def xmission_lolp_after(regional_hourlycaps,max_flows,return_eue=False):
    #,return_deficit_hourly_gen=False):    
    all_regions = ['CAMX', 'Desert_Southwest', 'NWPP_Central', 'NWPP_NE', 'NWPP_NW']
        
    residual_all = np.empty((0,n_iters,
                            n_hours))

    for region_here in all_regions:
        residual = regional_hourlycaps[region_here] - wecc_demand_year[region_here].to_numpy()
        residual_all = np.vstack((residual_all,[residual]))

    deficits = residual_all[:,np.any(residual_all<0,axis=0)]
    deficits = deficits.T

    deficit_locs = np.unique(np.argwhere(np.any(residual_all<0,axis=0)),axis=1)

    #print(deficit_locs.shape)
    residual_samples = da.array(deficits)
    
    model = init_model(max_flows)
    
    create_instance_getoutput_del = dask.delayed(create_instance_getoutput)

    output = [create_instance_getoutput_del(model,row) for row in residual_samples]

    #for idx,:
    #    
    #    output.append()
    start_time = time.perf_counter()

    result = dask.delayed(np.array)(output).compute(scheduler='processes')   

    end_time = time.perf_counter()

    print("time_taken: %0.2f mins"%((end_time - start_time)/60))
    
    deficit_array = np.zeros((len(all_regions),n_iters,8760))
    

    deficit_array[:,deficit_locs[:,0],deficit_locs[:,1]] = -1*(np.stack(result[:,1])).T

    lolp_df_afterxfers = pd.DataFrame((np.sum((deficit_array<0),axis=1)/n_iters).T,
                                      index=wecc_demand_year.index,columns=all_regions)

    eue_df_afterxfers = pd.DataFrame((np.sum(np.where(deficit_array<0,deficit_array,0),
                                              axis=1)/n_iters).T,
                                  index=wecc_demand_year.index,columns=all_regions)
    if return_eue:
        return (lolp_df_afterxfers,eue_df_afterxfers)
#    print('after\n',lolp_df_afterxfers.sum())
    
    return (lolp_df_afterxfers,deficit_array)

# Get LOLP new fleet

In [9]:
year_all = [str(year) for year in range(2017,2018)]#'2018'
RE_usage_all = ['45','60']
prm = '13'
ret = '0'

num_iters_sim = int('250')
n_iters = num_iters_sim
n_hours = 8760

for year in year_all:
    if calendar.isleap(int(year)):
        reqd_slice_year = slice(year+'-01-01',year+'-12-30')
    else:
        reqd_slice_year = year

    for RE_usage in RE_usage_all:

        # %%
        if calendar.isleap(int(year)):
            reqd_slice_year = slice(year+'-01-01',year+'-12-30')
        else:
            reqd_slice_year = year

        ce_dir = 'ResultsWECCC100'+'RE'+RE_usage+'PRM'+prm+'Yr'+year+'Ret'+ret+'/'
        output_fname = 'RE'+RE_usage+'PRM'+prm+'Yr'+year+'Ret'+ret
        print(output_fname)

        if RE_usage == '13':
            wecc_demand = pd.read_csv(ce_outputs_parent_dir+'ResultsWECCC100'+'RE'+'30'+'PRM13Yr'+year+'Ret0/'
                                      +'2022/demandInitial2022.csv', index_col='date_time')
        else:
            wecc_demand = pd.read_csv(ce_outputs_parent_dir+ce_dir+'2030/CE/demandFullYr2030.csv', 
                                      index_col='date_time')
        wecc_demand.index = pd.to_datetime(wecc_demand.index) #- np.timedelta64(8, 'h')
        wecc_demand_year = wecc_demand.loc[reqd_slice_year]

        wecc_hydro_monthlygen = pd.read_csv('hydro_monthly_totals.csv',index_col=0)
        
        all_regions = ['CAMX', 'Desert_Southwest', 'NWPP_Central', 'NWPP_NE', 'NWPP_NW']
        transmission_limits = pd.DataFrame(np.zeros((5,5)),index=all_regions,columns=all_regions)

        if RE_usage == '13':
            transmission_fromfile = pd.read_csv(ce_outputs_parent_dir+'ResultsWECCC100'+'RE'+'30'+
                                                'PRM13Yr'+year+'Ret0/'+'2030/CE/lineLimitsForCE2030.csv',
                                                index_col=0)
        else:
            transmission_fromfile = pd.read_csv(ce_outputs_parent_dir+ce_dir+
                                                '2030/CE/lineLimitsAfterCE2030.csv',index_col=0)


        for idx,row in transmission_fromfile.iterrows():
            transmission_limits.loc[row['r']][row['rr']] = row['TotalCapacity']

        max_flows = transmission_limits.to_numpy()
        max_flows = max_flows[~np.eye(max_flows.shape[0],dtype=bool)]

        with open("/nfs/turbo/seas-mtcraig/sriharis/paper1_revision_fleets/Mar22/modified_fleet_" \
                  +output_fname+".pkl","rb") as f:
            regional_caps_fors = pickle.load(f)

        lolp_df_beforexfers, regional_hourlycaps = get_lolp_system_wo_xmission(regional_caps_fors)

        lolp_df_afterxfers,eue_df_afterxfers = xmission_lolp_after(regional_hourlycaps, \
                                                                     max_flows,return_eue=True)

        print(lolp_df_afterxfers.sum())
        print(eue_df_afterxfers.sum())
        
        lolp_df_afterxfers.to_csv('../LOLP_eue/LOLP_'+output_fname+'.csv')
        eue_df_afterxfers.to_csv('../LOLP_eue/eue_'+output_fname+'.csv')


RE45PRM13Yr2019Ret0
time_taken: 223.24 mins
CAMX                1.944
Desert_Southwest    0.000
NWPP_Central        0.000
NWPP_NE             0.000
NWPP_NW             0.000
dtype: float64
CAMX               -829.566682
Desert_Southwest      0.000000
NWPP_Central          0.000000
NWPP_NE               0.000000
NWPP_NW               0.000000
dtype: float64
RE60PRM13Yr2019Ret0
time_taken: 323.19 mins
CAMX                2.56
Desert_Southwest    0.00
NWPP_Central        0.00
NWPP_NE             0.00
NWPP_NW             0.00
dtype: float64
CAMX               -1145.705604
Desert_Southwest       0.000000
NWPP_Central           0.000000
NWPP_NE                0.000000
NWPP_NW                0.000000
dtype: float64


# Iteration convergence

In [8]:
year_all = [str(year) for year in range(2017,2018)]#'2018'
RE_usage_all = ['45']
prm = '13'
ret = '0'

num_iters_sim = int('500')
n_iters = num_iters_sim
n_hours = 8760

for year in year_all:
    if calendar.isleap(int(year)):
        reqd_slice_year = slice(year+'-01-01',year+'-12-30')
    else:
        reqd_slice_year = year

    for RE_usage in RE_usage_all:

        # %%
        if calendar.isleap(int(year)):
            reqd_slice_year = slice(year+'-01-01',year+'-12-30')
        else:
            reqd_slice_year = year

        ce_dir = 'ResultsWECCC100'+'RE'+RE_usage+'PRM'+prm+'Yr'+year+'Ret'+ret+'/'
        output_fname = 'RE'+RE_usage+'PRM'+prm+'Yr'+year+'Ret'+ret

        if RE_usage == '13':
            wecc_demand = pd.read_csv(ce_outputs_parent_dir+'ResultsWECCC100'+'RE'+'30'+'PRM13Yr'+year+'Ret0/'
                                      +'2022/demandInitial2022.csv', index_col='date_time')
        else:
            wecc_demand = pd.read_csv(ce_outputs_parent_dir+ce_dir+'2030/CE/demandFullYr2030.csv', 
                                      index_col='date_time')

        wecc_demand.index = pd.to_datetime(wecc_demand.index) #- np.timedelta64(8, 'h')
        wecc_demand_year = wecc_demand.loc[reqd_slice_year]

        wecc_hydro_monthlygen = pd.read_csv('hydro_monthly_totals.csv',index_col=0)
        
        all_regions = ['CAMX', 'Desert_Southwest', 'NWPP_Central', 'NWPP_NE', 'NWPP_NW']
        transmission_limits = pd.DataFrame(np.zeros((5,5)),index=all_regions,columns=all_regions)

        if RE_usage == '13':
            transmission_fromfile = pd.read_csv(ce_outputs_parent_dir+'ResultsWECCC100'+'RE'+'30'+
                                                'PRM13Yr'+year+'Ret0/'+'2030/CE/lineLimitsForCE2030.csv',
                                                index_col=0)
        else:
            transmission_fromfile = pd.read_csv(ce_outputs_parent_dir+ce_dir+
                                                '2030/CE/lineLimitsAfterCE2030.csv',index_col=0)


        for idx,row in transmission_fromfile.iterrows():
            transmission_limits.loc[row['r']][row['rr']] = row['TotalCapacity']

        max_flows = transmission_limits.to_numpy()
        max_flows = max_flows[~np.eye(max_flows.shape[0],dtype=bool)]

        with open("/nfs/turbo/seas-mtcraig/sriharis/paper1_revision_fleets/Mar22/modified_fleet_" \
                  +output_fname+".pkl","rb") as f:
            regional_caps_fors = pickle.load(f)
        
        
        output_fstream = open('log_iterate_converge_RE'+RE_usage+'Yr'+year+'Iters'+str(n_iters)+'.txt','a')
        print('Trial number, LOLH, EUE, Time',file=output_fstream)
        output_fstream.flush()

        for trial_idx in range(16,25):
            
            trial_fname = 'iterate_converge/RE'+RE_usage+'Yr'+year+'Trial'+str(trial_idx)+'_'+str(n_iters)

            start_time = time.perf_counter()

            lolp_df_beforexfers, regional_hourlycaps = get_lolp_system_wo_xmission(regional_caps_fors)

            lolp_df_afterxfers,eue_df_afterxfers = xmission_lolp_after(regional_hourlycaps, \
                                                                         max_flows,return_eue=True)

            end_time = time.perf_counter()

            time_taken = (end_time - start_time)/60

            print(lolp_df_afterxfers['CAMX'].sum(),eue_df_afterxfers['CAMX'].sum(),"%0.2f"%(time_taken))
            
            print(str(trial_idx)+','+\
                  str(lolp_df_afterxfers['CAMX'].sum())+','+\
                  str(eue_df_afterxfers['CAMX'].sum())+','+\
                  "%0.2f"%(time_taken),
                  file=output_fstream)
            output_fstream.flush()

            lolp_df_afterxfers.to_csv(trial_fname+'_lolp.csv')
            eue_df_afterxfers.to_csv(trial_fname+'_eue.csv')


time_taken: 71.31 mins
2.422 -2957.1455880924536 74.49
time_taken: 69.73 mins
2.37 -2908.2154940702235 71.75
time_taken: 69.92 mins
2.4019999999999997 -2911.847847208605 71.97
time_taken: 69.19 mins
2.4019999999999997 -2930.9664679617395 71.17
time_taken: 70.50 mins
2.388 -2876.7826322187593 73.04
time_taken: 70.12 mins
2.388 -2867.716915190575 72.17
time_taken: 70.55 mins
2.3999999999999995 -2884.3512528309016 72.54
time_taken: 69.09 mins
2.4059999999999997 -2949.026725951962 71.22
time_taken: 69.54 mins
2.388 -2892.8250571646468 71.55


# Run specific cases and modify capacity

In [9]:
year_all = [str(year) for year in range(2019,2020)]#'2018'
RE_usage_all = ['45']
prm = '13'
ret = '0'

num_iters_sim = int('50')
n_iters = num_iters_sim
n_hours = 8760

output_fstream = open('log_19.txt','a')
print('Start',file=output_fstream)

for year in year_all:
    if calendar.isleap(int(year)):
        reqd_slice_year = slice(year+'-01-01',year+'-12-30')
    else:
        reqd_slice_year = year

    for RE_usage in RE_usage_all:

        # %%
        if calendar.isleap(int(year)):
            reqd_slice_year = slice(year+'-01-01',year+'-12-30')
        else:
            reqd_slice_year = year

        ce_dir = 'ResultsWECCC100'+'RE'+RE_usage+'PRM'+prm+'Yr'+year+'Ret'+ret+'/'
        output_fname = 'RE'+RE_usage+'PRM'+prm+'Yr'+year+'Ret'+ret
        print(output_fname)

        if RE_usage == '13':
            wecc_demand = pd.read_csv(ce_outputs_parent_dir+'ResultsWECCC100'+'RE'+'30'+'PRM13Yr'+year+'Ret0/'
                                      +'2022/demandInitial2022.csv', index_col='date_time')
        else:
            wecc_demand = pd.read_csv(ce_outputs_parent_dir+ce_dir+'2030/CE/demandFullYr2030.csv', index_col='date_time')
        wecc_demand.index = pd.to_datetime(wecc_demand.index) #- np.timedelta64(8, 'h')
        wecc_demand_year = wecc_demand.loc[reqd_slice_year]

        wecc_hydro_monthlygen = pd.read_csv('hydro_monthly_totals.csv',index_col=0)

        #### RAM with MC fleets
        # %%
        if RE_usage == '13':
            fleet = pd.read_csv(ce_outputs_parent_dir+'ResultsWECCC100'+'RE'+'30'+'PRM13Yr'+year+'Ret0/'
                                +'2030/CE/genFleetForCEPreRECombine2030.csv',index_col=0)
        else:
            fleet = pd.read_csv(ce_outputs_parent_dir+ce_dir
                                +'2030/CE/genFleetAfterCEForRAMUncompressed.csv',index_col=0)
        start_time = time.perf_counter()

        regional_caps_fors = get_subregional_fleets(fleet,wecc_demand_year)

        lolp_df_beforexfers, regional_hourlycaps = get_lolp_system_wo_xmission(regional_caps_fors)

        all_regions = ['CAMX', 'Desert_Southwest', 'NWPP_Central', 'NWPP_NE', 'NWPP_NW']

        transmission_limits = pd.DataFrame(np.zeros((5,5)),index=all_regions,columns=all_regions)
        
        if RE_usage == '13':
            transmission_fromfile = pd.read_csv(ce_outputs_parent_dir+'ResultsWECCC100'+'RE'+'30'+'PRM13Yr'+year+'Ret0/'
                                                '2030/CE/lineLimitsForCE2030.csv',index_col=0)
        else:
            transmission_fromfile = pd.read_csv(ce_outputs_parent_dir+ce_dir+
                                                '2030/CE/lineLimitsAfterCE2030.csv',index_col=0)


        for idx,row in transmission_fromfile.iterrows():
            transmission_limits.loc[row['r']][row['rr']] = row['TotalCapacity']

        max_flows = transmission_limits.to_numpy()
        max_flows = max_flows[~np.eye(max_flows.shape[0],dtype=bool)]

        #max_flows[3] = 2000
        #max_flows[15] = 5000
        #max_flows[16] = 2000
        #max_flows[19] = 5000

        # %%
        lolp_df_afterxfers,deficit_array = xmission_lolp_after(regional_hourlycaps, \
                                                                     max_flows)

        ## Modify fleet

        reqd_region = 'CAMX'
        caps_for_region = regional_caps_fors[reqd_region].copy()

        nameplate_region = caps_for_region['nameplate_region']
        tech_region = caps_for_region['tech_region']
        CC_region = nameplate_region.loc[tech_region['PlantType'].isin(['CC'])]

        ### Remove capacity

        #remove_cap = 4750
        while(lolp_df_afterxfers['CAMX'].sum() < 2.3):

            remove_cap = 1000

            nameplate_region = caps_for_region['nameplate_region']
            tech_region = caps_for_region['tech_region']
            CC_region = nameplate_region.loc[tech_region['PlantType'].isin(['CC'])]
            for_matrix = caps_for_region['for_matrix']
            hourly_capacity = caps_for_region['hourly_capacity']

            drop_indexes = CC_region.loc[(CC_region.sort_values().cumsum()<remove_cap)].index

            nameplate_region = nameplate_region.drop(drop_indexes,axis=0).reset_index(drop=True)
            tech_region = tech_region.drop(drop_indexes,axis=0).reset_index(drop=True)

            caps_for_region['nameplate_region'] = nameplate_region
            caps_for_region['tech_region'] = tech_region
            caps_for_region['hourly_capacity'] = da.from_array(np.delete(caps_for_region['hourly_capacity'].compute(),
                                                                         drop_indexes,1),
                                                               chunks=(150,nameplate_region.size))
            caps_for_region['hourly_capacity_factors'] = da.from_array(np.delete(caps_for_region['hourly_capacity_factors'].compute(),
                                                                         drop_indexes,1),
                                                               chunks=(150,nameplate_region.size))
            caps_for_region['for_matrix'] = da.from_array(np.delete(caps_for_region['for_matrix'].compute(),
                                                                         drop_indexes,1),
                                                               chunks=(150,nameplate_region.size))

            lolp_df, regional_hourlycaps_region = get_lolp_system_wo_xmission({reqd_region:caps_for_region},\
                                                                                         reqd_region)
            regional_hourlycaps[reqd_region] = regional_hourlycaps_region[reqd_region]

            # %%
            lolp_df_afterxfers,deficit_array = xmission_lolp_after(regional_hourlycaps, \
                                                                   max_flows)

        ### Add capacity

        while (lolp_df_afterxfers[reqd_region].sum() > 2.5):

            caps_for_region_copy = caps_for_region.copy()

            cap_added = 1000
            num_added = np.floor(cap_added/CC_region.median())
            addition = np.repeat(CC_region.median(),num_added)
            addition = np.append(addition, np.ceil(cap_added%CC_region.median()))
            for_added = da.ones((8760,len(addition)))*0.05

            nameplate_region = pd.concat([nameplate_region,pd.Series(addition,
                                                                        name=nameplate_region.name)
                                    ]).reset_index(drop=True)
            tech_region = pd.concat([tech_region,pd.DataFrame(['Other']*len(addition),
                                                              columns=['PlantType'])
                                    ]).reset_index(drop=True)

            for_matrix = da.hstack((caps_for_region['for_matrix'],for_added))
            hourly_capacity_added = da.ones_like(for_added)*addition
            hourly_capacity = da.hstack((caps_for_region['hourly_capacity'],hourly_capacity_added))

            for_matrix = for_matrix.compute()
            hourly_capacity = hourly_capacity.compute()

            for_matrix = da.from_array(for_matrix,chunks=(150,nameplate_region.size))
            hourly_capacity = da.from_array(hourly_capacity,chunks=(150,nameplate_region.size))

            caps_for_region['nameplate_region'] = nameplate_region
            caps_for_region['tech_region'] = tech_region
            caps_for_region['hourly_capacity'] = hourly_capacity
            caps_for_region['for_matrix'] = for_matrix

            lolp_df, regional_hourlycaps_region = get_lolp_system_wo_xmission({reqd_region:caps_for_region},\
                                                                                     reqd_region)
            regional_hourlycaps[reqd_region] = regional_hourlycaps_region[reqd_region]

            # %%
            lolp_df_afterxfers,deficit_array = xmission_lolp_after(regional_hourlycaps, \
                                                                   max_flows)



            if(lolp_df_afterxfers[reqd_region].sum() > 2.5):
                continue
            else:

                lolh = lolp_df_afterxfers[reqd_region].sum()
                capacity_try_1 = 1000
                capacity_try_2 = 0
                num = 0

                while (np.abs(lolh - 2.4) > 5e-2) & (capacity_try_1 - capacity_try_2 > 100):

                    caps_for_region = caps_for_region_copy.copy()
                    nameplate_region = caps_for_region['nameplate_region']
                    tech_region = caps_for_region['tech_region']
                    CC_region = nameplate_region.loc[tech_region['PlantType'].isin(['CC'])]
                    for_matrix = caps_for_region['for_matrix']
                    hourly_capacity = caps_for_region['hourly_capacity']

                    cap_added = (capacity_try_1 + capacity_try_2)/2
                    num_added = np.floor(cap_added/CC_region.median())
                    addition = np.repeat(CC_region.median(),num_added)
                    addition = np.append(addition, np.ceil(cap_added%CC_region.median()))
                    for_added = da.ones((8760,len(addition)))*0.05

                    nameplate_region = pd.concat([nameplate_region,pd.Series(addition,
                                                                        name=nameplate_region.name)
                                                 ]).reset_index(drop=True)
                    tech_region = pd.concat([tech_region,pd.DataFrame(['Other']*len(addition),
                                                                      columns=['PlantType'])
                                            ]).reset_index(drop=True)

                    for_matrix = da.hstack((caps_for_region['for_matrix'],for_added))
                    hourly_capacity_added = da.ones_like(for_added)*addition
                    hourly_capacity = da.hstack((caps_for_region['hourly_capacity'],hourly_capacity_added))

                    for_matrix = for_matrix.compute()
                    hourly_capacity = hourly_capacity.compute()

                    for_matrix = da.from_array(for_matrix,chunks=(150,nameplate_region.size))
                    hourly_capacity = da.from_array(hourly_capacity,chunks=(150,nameplate_region.size))

                    caps_for_region['nameplate_region'] = nameplate_region
                    caps_for_region['tech_region'] = tech_region
                    caps_for_region['hourly_capacity'] = hourly_capacity
                    caps_for_region['for_matrix'] = for_matrix

                    lolp_df, regional_hourlycaps_region = get_lolp_system_wo_xmission({reqd_region:caps_for_region},\
                                                                                             reqd_region)
                    regional_hourlycaps[reqd_region] = regional_hourlycaps_region[reqd_region]

                    # %%
                    lolp_df_afterxfers,deficit_array = xmission_lolp_after(regional_hourlycaps, \
                                                                           max_flows)

                    lolh = lolp_df_afterxfers[reqd_region].sum()

                    if lolh - (2.4-1e-4) < 0:
                        capacity_try_1 = cap_added
                    elif lolh - (2.4-1e-4) > 0:
                        capacity_try_2 = cap_added

                    num +=1        

        ### RAM after modification

        lolp_df, regional_hourlycaps_region = get_lolp_system_wo_xmission({reqd_region:caps_for_region},\
                                                                                     reqd_region)
        regional_hourlycaps[reqd_region] = regional_hourlycaps_region[reqd_region]

        lolp_df.sum()

        # %%
        lolp_df_afterxfers,deficit_array = xmission_lolp_after(regional_hourlycaps, \
                                                               max_flows)

        ## Rerun with new fleet

        regional_caps_fors[reqd_region] = caps_for_region

        n_iters = 250
        lolp_df_beforexfers, regional_hourlycaps = get_lolp_system_wo_xmission(regional_caps_fors)

        all_regions = ['CAMX', 'Desert_Southwest', 'NWPP_Central', 'NWPP_NE', 'NWPP_NW']

        transmission_limits = pd.DataFrame(np.zeros((5,5)),index=all_regions,columns=all_regions)
        if RE_usage == '13':
            transmission_fromfile = pd.read_csv(ce_outputs_parent_dir+'ResultsWECCC100'+'RE'+'30'+'PRM13Yr'+year+'Ret0/'
                                                '2030/CE/lineLimitsForCE2030.csv',index_col=0)
        else:
            transmission_fromfile = pd.read_csv(ce_outputs_parent_dir+ce_dir+
                                                '2030/CE/lineLimitsAfterCE2030.csv',index_col=0)


        for idx,row in transmission_fromfile.iterrows():
            transmission_limits.loc[row['r']][row['rr']] = row['TotalCapacity']

        max_flows = transmission_limits.to_numpy()
        max_flows = max_flows[~np.eye(max_flows.shape[0],dtype=bool)]

        #max_flows[3] = 2000
        #max_flows[15] = 5000
        #max_flows[16] = 2000
        #max_flows[19] = 5000

        # %%
        lolp_df_afterxfers,deficit_array = xmission_lolp_after(regional_hourlycaps, \
                                                                     max_flows)

        with open("/nfs/turbo/seas-mtcraig/sriharis/paper1_revision_fleets/Mar22/modified_fleet_"+output_fname+".pkl","wb") as f:
            pickle.dump(regional_caps_fors, f, pickle.HIGHEST_PROTOCOL)

        lolp_df_afterxfers.to_csv('LOLP_'+output_fname+'.csv')

        end_time = time.perf_counter()

        time_taken = (end_time - start_time)/60

        print("Case:"+output_fname+\
              ",LOLH=%0.2f"%(lolp_df_afterxfers[reqd_region].sum())\
              +",time taken:%0.2f"%(time_taken),
              file=output_fstream)

        output_fstream.flush()

RE45PRM13Yr2019Ret0
time_taken: 1.62 mins
time_taken: 1.54 mins
time_taken: 1.73 mins
time_taken: 2.09 mins
time_taken: 2.58 mins
time_taken: 3.06 mins
time_taken: 3.76 mins
time_taken: 4.53 mins
time_taken: 5.44 mins
time_taken: 6.98 mins
time_taken: 8.79 mins
time_taken: 10.68 mins
time_taken: 12.98 mins
time_taken: 10.44 mins
time_taken: 11.77 mins
time_taken: 10.84 mins
time_taken: 10.74 mins
time_taken: 10.86 mins
time_taken: 8.42 mins
time_taken: 9.68 mins
time_taken: 10.31 mins
time_taken: 10.63 mins
time_taken: 10.69 mins
time_taken: 10.81 mins
time_taken: 225.86 mins


In [1]:
#11.35